<a href="https://colab.research.google.com/github/stellarsb7/GFS-Net/blob/main/Source%20Code/MacroscopicOralCancerIdentificationModelNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

! mkdir ~/.kaggle

!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

! chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d sumanghosh1/oral-cancer-dataset2-split-and-augmented

In [ ]:
! unzip oral-cancer-dataset2-split-and-augmented.zip

In [ ]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import tensorflow as tf
from keras.metrics import Accuracy
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow import keras
from tensorflow.keras.utils import plot_model
#from keras import backend as K

import os
import cv2
from PIL import Image
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical

!pip install Py_FS
!pip install ReliefF
from  Py_FS.evaluation.evaluate import evaluate

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
#@title Training Dataset
image_directory ="/content/Oral_Cancer_Split"
SIZE = 224
X_train = []
y_train = []

cancerous = os.listdir(os.path.join(image_directory,''))
for i, image_name in enumerate(cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

    if (image_name.split('.')[1] == 'jpg'):
        path=os.path.join(image_directory,'OSCC',image_name)
        # print(path)
        image = cv2.imread(path)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        X_train.append(np.array(image))
        y_train.append(1)

non_cancerous = os.listdir(os.path.join(image_directory,'Normal'))
for i, image_name in enumerate(non_cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(os.path.join(image_directory,'Normal',image_name))
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        X_train.append(np.array(image))
        y_train.append(0)


image_directory = '/content/val/'
SIZE = 224

cancerous = os.listdir(os.path.join(image_directory , 'OSCC'))
for i, image_name in enumerate(cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(os.path.join(image_directory,'OSCC',image_name))
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        X_train.append(np.array(image))
        y_train.append(1)

non_cancerous = os.listdir(os.path.join(image_directory,'Normal'))
for i, image_name in enumerate(non_cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(os.path.join(image_directory,'Normal',image_name))
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        X_train.append(np.array(image))
        y_train.append(0)


print(len(X_train))
print(len(y_train))

In [ ]:
#@title Test Dataset
image_directory = '/content/test/'
SIZE = 224
X_test = []
y_test = []

cancerous = os.listdir(os.path.join(image_directory , 'OSCC'))
for i, image_name in enumerate(cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(os.path.join(image_directory,'OSCC',image_name))
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        X_test.append(np.array(image))
        y_test.append(1)

non_cancerous = os.listdir(os.path.join(image_directory,'Normal'))
for i, image_name in enumerate(non_cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(os.path.join(image_directory,'Normal',image_name))
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        X_test.append(np.array(image))
        y_test.append(0)

print(len(X_test))
print(len(y_test))

In [ ]:
def se_block(input_tensor, reduction_ratio=16):
    """
    Squeeze-and-Excitation block implementation.

    Args:
        input_tensor: Input tensor to the SE block.
        reduction_ratio: Reduction ratio for the number of parameters in the SE block.
                         It controls the number of parameters used in the bottleneck layer.

    Returns:
        Output tensor after applying the SE block.
    """
    channels = input_tensor.shape[-1]

    # Squeeze operation: Global Average Pooling
    squeeze = tf.keras.layers.GlobalAveragePooling2D()(input_tensor)

    # Excitation: Fully connected layers
    excitation = tf.keras.layers.Dense(channels // reduction_ratio, activation='relu')(squeeze)
    excitation = tf.keras.layers.Dense(channels, activation='sigmoid')(excitation)

    # Reshape to match the shape of the input tensor
    excitation = tf.keras.layers.Reshape((1, 1, channels))(excitation)

    # Scale the input tensor by the excitation signal
    scaled_input = input_tensor * excitation

    return scaled_input

In [ ]:
macroscopic_model_nosq = tf.keras.models.load_model('/content/drive/MyDrive/SugataSaptarshi/Oral Cancer macroscopic/macroscopicnasnetnosq (1).keras')


In [ ]:
modelsq.evaluate(np.array(X_test), to_categorical(np.array(y_test)))

In [ ]:
modelnosq = tf.keras.models.load_model('/content/drive/MyDrive/SugataSaptarshi/ORCC histopathology/NasNetMobile/models/nasnet.keras')


In [ ]:
modelnosq.evaluate(np.array(X_test), to_categorical(np.array(y_test)))

In [ ]:
modelsq.summary()

In [ ]:
modelnosq.summary()